In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [94]:
df = pd.read_csv("Churn_Modelling.csv") 

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [95]:
df = df.drop(["RowNumber", "CustomerId","Surname"], axis=1)

In [96]:
label_encoder_gender = LabelEncoder()
df["Gender"]= label_encoder_gender.fit_transform(df["Gender"])
df.head()



,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [97]:
from sklearn.preprocessing import OneHotEncoder
OHE_geo = OneHotEncoder()  #designating our class
OHE_geo_data= OHE_geo.fit_transform(df[['Geography']])
OHE_geo_data

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [98]:
geo_feats = OHE_geo.get_feature_names_out(['Geography'])

In [99]:
OHEGeoData_df = pd.DataFrame(OHE_geo_data.toarray(), columns= geo_feats)
OHEGeoData_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [100]:
df= pd.concat([df.drop("Geography",axis=1), OHEGeoData_df],axis=1)

In [101]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [102]:
#Independent and Dependent

X = df.drop('Exited',axis=1)
y = df["Exited"]

#Splitting the data into training and testing data
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=42)

#Scaling the independent data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  =  scaler.transform(X_test)

In [103]:
#Filing Objects in pickle file


with open("scaler.pkl","wb") as file:
    pickle.dump(scaler, file)

with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("OHE_geo.pkl",'wb')as file:
    pickle.dump(OHE_geo,file)


ANN TRAINING

In [104]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [105]:
#Model Creation

model = Sequential([
Dense(64,activation="relu",input_shape=(X_train.shape[1],)),
Dense(32,activation="relu"),
Dense(1,activation="sigmoid")

])

c:\Users\MTHA\Downloads\annclassification (2)\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [106]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [107]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#Create LOGS directory including datetime & call and assign TENSORBOARD 

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

NameError: name 'datetime' is not defined

In [109]:
#EarlyStopping
earlystopping_callback = EarlyStopping(monitor="val_loss", patience= 5, restore_best_weights=True)

In [118]:
#Model Training

history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,earlystopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8662 - loss: 0.3174 - val_accuracy: 0.8610 - val_loss: 0.3360
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.8621 - loss: 0.3308 - val_accuracy: 0.8655 - val_loss: 0.3391
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8709 - loss: 0.3095 - val_accuracy: 0.8590 - val_loss: 0.3432
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8739 - loss: 0.3124 - val_accuracy: 0.8605 - val_loss: 0.3397
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8637 - loss: 0.3285 - val_accuracy: 0.8610 - val_loss: 0.3378
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8623 - loss: 0.3217 - val_accuracy: 0.8645 - val_loss: 0.3388


In [119]:
model.save("model.h5")

In [120]:
#Load tensorboard Extension
%reload_ext tensorboard


In [121]:
%tensorboard --logdir logs/fit/20250601-173408



Reusing TensorBoard on port 6006 (pid 12520), started 1:42:56 ago. (Use '!kill 12520' to kill it.)